In [2]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 17 13:11:12 2023

@author: Debra Hogue

Modified RankNet by Lv et al. to use Tensorflow not Pytorch
and added additional comments to explain methods

Paper: Simultaneously Localize, Segment and Rank the Camouflaged Objects by Lv et al.
"""

import tensorflow as tf
from tensorflow.keras import activations, layers, losses
import numpy as np
import os, argparse
from datetime import datetime
from Attention.ResNet_models import Generator
from data import get_loader
from utils import adjust_lr, AvgMeter
from scipy import misc
import cv2
from data import test_dataset
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2
import tensorflow.keras.applications.resnet50 as models # instantiates the ResNet50 architecture 

from utils import l2_regularisation
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
gpus = tf.config.list_physical_devices('GPU')
try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=5632)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

tf.config.optimizer.set_experimental_options({"Memory optimizer": True, "Layout optimizer": True})
tf.keras.mixed_precision.set_global_policy('mixed_float16')

2023-06-01 09:28:14.772259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-01 09:28:15.521469: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


1 Physical GPUs, 1 Logical GPUs
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: Quadro RTX 3000, compute capability 7.5


2023-06-01 09:28:17.462335: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-01 09:28:17.502220: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-01 09:28:17.502289: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-01 09:28:17.504690: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-01 09:28:17.504752: I tensorflow/compile

In [4]:

# parser = argparse.ArgumentParser()
# parser.add_argument('--epoch', type=int, default=1, help='epoch number')
# parser.add_argument('--lr_gen', type=float, default=2.5e-5, help='learning rate for generator')
# parser.add_argument('--batchsize', type=int, default=2, help='training batch size')
# parser.add_argument('--trainsize', type=int, default=480, help='training dataset size')
# parser.add_argument('--decay_rate', type=float, default=0.9, help='decay rate of learning rate')
# parser.add_argument('--decay_epoch', type=int, default=40, help='every n epochs decay learning rate')
# parser.add_argument('--feat_channel', type=int, default=32, help='reduced channel of saliency feat')
# opt = parser.parse_args()
# print('Generator Learning Rate: {}'.format(opt.lr_gen))
epoch = 3
decay_rate = 0.97
decay_epoch = 65
batchsize = 3

In [5]:

# build models
generator = Generator(channel=32)


# generator_params = generator.parameters()
# generator_optimizer = tf.optimizers.Adam(generator_params, opt.lr_gen)


image_root = '../dataset/train/Imgs/'
gt_root = '../dataset/train/GT/'
fix_root = '../dataset/train/Fix/'

# train_loader = get_loader(image_root, gt_root, fix_root,batchsize=opt.batchsize, trainsize=opt.trainsize)
# total_step = len(train_loader)

CE = losses.BinaryCrossentropy(from_logits=True)
mse_loss = losses.MeanSquaredError()
size_rates = [0.75,1,1.25]  # multi-scale training

def structure_loss(pred, mask):
    padded = tf.pad(mask,tf.constant([[0,0],[15,15],[15,15],[0,0]]))
    pooled =tf.nn.avg_pool2d(padded, ksize=31, strides=1, padding="VALID")
    weit  = 1+5*tf.abs(pooled-mask)
    weit = tf.squeeze(weit,[3])
    wbce= tf.nn.sigmoid_cross_entropy_with_logits(mask,pred)
   
    wbce= tf.math.reduce_mean(wbce)
    wbce  = tf.math.reduce_sum((weit*wbce),axis=[1,2]) /tf.reduce_sum(weit,axis=[1,2])
    mask =tf.squeeze(mask,[3])
    pred  = tf.math.sigmoid(pred)
    pred = tf.squeeze(pred,[3])
    inter = tf.math.reduce_sum((pred*mask)*weit,axis=[1,2])
    union = tf.math.reduce_sum((pred+mask)*weit, axis=[1,2])
    wiou  = 1-(inter+1)/(union-inter+1)
    return tf.math.reduce_mean(wbce+wiou)


        
             
def loss_function(y_true,y_pred):
    gts, fixs = tf.unstack(y_true,2,0)
    gts, _ = tf.split(gts, [1,2], 3)
    fixs, _ = tf.split(fixs, [1,2], 3)
    fix_pred, cod_pred1, cod_pred2 = tf.unstack(y_pred,num=3,axis=0)
    fix_loss = mse_loss(tf.keras.activations.sigmoid(fix_pred),fixs)
    cod_loss1 = structure_loss(cod_pred1, gts)
    cod_loss2 = structure_loss(cod_pred2, gts)
    test= fix_loss + cod_loss1 + cod_loss2
    return  fix_loss + cod_loss1 + cod_loss2
    
def on_epoch_end( epoch, lr):
    decay = decay_rate ** (epoch // decay_epoch)
    new_lr = lr * decay
    print("\nEpoch: {}. Reducing Learning Rate from {} to {}".format(epoch, lr, new_lr))
    return new_lr
        





2023-06-01 09:28:26.829479: I tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:736] failed to allocate 5.50GiB (5905580032 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory
2023-06-01 09:28:27.204463: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2023-06-01 09:28:27.204515: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2023-06-01 09:28:27.204621: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1679] Profiler found 1 GPUs
2023-06-01 09:28:27.227431: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_error_manager.cc:194] cuptiSubscribe: error 15: CUPTI_ERROR_NOT_INITIALIZED
2023-06-01 09:28:27.227497: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_error_manager.cc:459] cuptiGetResultString: ignored due to a previous error.
2023-06-01 09:28:27.227508: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1730] function cupti_interface_->Subscribe( &subscribe

2001


2023-06-01 09:28:52.727464: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8901
2023-06-01 09:28:53.785126: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


KeyboardInterrupt: 

In [ ]:
        
if __name__ == '__main__':
    
  
    
    
    
    
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, profile_batch='10, 15')
    
    op= tf.keras.optimizers.Adam(learning_rate=2.5e-5, name='Adam')
    
    generator.compile(optimizer=op, loss=loss_function, run_eagerly=False)
    
    
    data = get_loader(image_root, gt_root, fix_root, 480, batchsize, size_rates)
    print(len(data))
    generator.fit(x=data,batch_size=batchsize, epochs=epoch, verbose='auto' , callbacks=[tensorboard_callback, tf.keras.callbacks.LearningRateScheduler(on_epoch_end)])
     
     

In [7]:

save_path = 'models/Resnet/'
    





dataset_path = '../dataset/test/'

In [ ]:
if not os.path.exists(save_path):
        os.makedirs(save_path)
print("Generator is build")
generator.save("results/model")

Generator is build


2023-06-01 08:23:44.637902: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-06-01 08:23:44.667520: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-06-01 08:23:44.678365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-06-01 08

2023-06-01 08:23:45.596991: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-06-01 08:23:45.613814: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]


2023-06-01 08:23:55.590384: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'saliency_feat_encoder/989415' with dtype float and shape [31,31,1,1]
	 [[{{node saliency_feat_encoder/989415}}]]
2023-06-01 08:23:57.137246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'saliency_feat_encoder/994613' with dtype float and shape [31,31,1,1]
	 [[{{node saliency_feat_encoder/994613}}]]
2023-06-01 08:23:57.341556: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for place

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 2048, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f02980e2370>, 139651935273008), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f02981165e0>, 139651935273168), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 2048, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f029809ab80>, 139651935273328), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f02980b7c10>, 139651935273488), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3

...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 2048, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f02980e2370>, 139651935273008), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f02981165e0>, 139651935273168), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3, 3, 2048, 32), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f029809ab80>, 139651935273328), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(32,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f02980b7c10>, 139651935273488), {}).
INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(3

2023-06-01 08:24:08.322367: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-06-01 08:24:10.945612: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-06-01 08:24:10.956348: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype half and shape [?,?,?,?]
	 [[{{node inputs}}]]
2023-06-01 08

2023-06-01 08:24:18.173896: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'serving_default_input_1' with dtype half and shape [?,352,352,3]
	 [[{{node serving_default_input_1}}]]


INFO:tensorflow:Assets written to: results/model/assets


INFO:tensorflow:Assets written to: results/model/assets


In [9]:
test_datasets = ['Mine', 'CAMO']


In [5]:

for dataset in test_datasets:
    save_path = './results/ResNet50/' + dataset + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    image_root = dataset_path + dataset + '/Imgs/'
    test_loader = test_dataset(image_root, 480)

    for i in range(test_loader.size):
        print(i)
        image, HH, WW, name = test_loader.load_data()
        ans = generator(image)
        _,generator_pred, _  = tf.unstack(ans,num=3,axis=0)
        res = generator_pred
        res = tf.image.resize(res, size=tf.constant([WW,HH]), method=tf.image.ResizeMethod.BILINEAR)
        res = tf.math.sigmoid(res).numpy().squeeze()
        res = 255*(res - res.min()) / (res.max() - res.min() + 1e-8)
        print(save_path+name)
        cv2.imwrite(save_path+name, res)
        print()

../dataset/test/Mine/Imgs/
['camourflage_00308.jpg', 'camourflage_00320.jpg', 'camourflage_00326.jpg', 'camourflage_00349.jpg', 'camourflage_00369.jpg', 'camourflage_00375.jpg', 'camourflage_00382.jpg', 'camourflage_00387.jpg', 'camourflage_00409.jpg', 'camourflage_00413.jpg', 'camourflage_00417.jpg', 'camourflage_00426.jpg', 'camourflage_00431.jpg', 'camourflage_00440.jpg', 'camourflage_00445.jpg', 'camourflage_00506.jpg', 'camourflage_00526.jpg', 'camourflage_00530.jpg', 'camourflage_00565.jpg', 'camourflage_00587.jpg', 'camourflage_00588.jpg', 'camourflage_00590.jpg', 'camourflage_00655.jpg', 'camourflage_00708.jpg', 'camourflage_00714.jpg', 'camourflage_00719.jpg', 'camourflage_00748.jpg', 'camourflage_00760.jpg', 'camourflage_00767.jpg', 'camourflage_00769.jpg', 'camourflage_00871.jpg', 'camourflage_00962.jpg', 'camourflage_00968.jpg', 'camourflage_00983.jpg', 'camourflage_01043.jpg', 'camourflage_01120.jpg', 'camourflage_01130.jpg', 'camourflage_01176.jpg', 'camourflage_01207.jpg

In [10]:

#generator2 = Generator(channel=32)
generator2 =  tf.keras.models.load_model('./results/FACE-100/', custom_objects={'loss_function': loss_function})
for dataset in test_datasets:
    save_path = './results/small/' + dataset + '/'
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    image_root = dataset_path + dataset + '/Imgs/'
    test_loader = test_dataset(image_root, 480)

    for i in range(test_loader.size):
        print(i)
        image, HH, WW, name = test_loader.load_data()
        ans = generator2(image)
        _,generator_pred, _  = tf.unstack(ans,num=3,axis=0)
        res = generator_pred
        res = tf.image.resize(res, size=tf.constant([WW,HH]), method=tf.image.ResizeMethod.BILINEAR)
        res = tf.math.sigmoid(res).numpy().squeeze()
        res = 255*(res - res.min()) / (res.max() - res.min() + 1e-8)
        print(save_path+name)
        cv2.imwrite(save_path+name, res)
        print()

../dataset/test/Mine/Imgs/
['camourflage_00308.jpg', 'camourflage_00320.jpg', 'camourflage_00326.jpg', 'camourflage_00349.jpg', 'camourflage_00369.jpg', 'camourflage_00375.jpg', 'camourflage_00382.jpg', 'camourflage_00387.jpg', 'camourflage_00409.jpg', 'camourflage_00413.jpg', 'camourflage_00417.jpg', 'camourflage_00426.jpg', 'camourflage_00431.jpg', 'camourflage_00440.jpg', 'camourflage_00445.jpg', 'camourflage_00506.jpg', 'camourflage_00526.jpg', 'camourflage_00530.jpg', 'camourflage_00565.jpg', 'camourflage_00587.jpg', 'camourflage_00588.jpg', 'camourflage_00590.jpg', 'camourflage_00655.jpg', 'camourflage_00708.jpg', 'camourflage_00714.jpg', 'camourflage_00719.jpg', 'camourflage_00748.jpg', 'camourflage_00760.jpg', 'camourflage_00767.jpg', 'camourflage_00769.jpg', 'camourflage_00871.jpg', 'camourflage_00962.jpg', 'camourflage_00968.jpg', 'camourflage_00983.jpg', 'camourflage_01043.jpg', 'camourflage_01120.jpg', 'camourflage_01130.jpg', 'camourflage_01176.jpg', 'camourflage_01207.jpg

ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * <tf.Tensor 'x:0' shape=(1, 480, 480, 3) dtype=float16>
  Keyword arguments: {'training': False}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (1 total):
    * TensorSpec(shape=(None, 352, 352, 3), dtype=tf.float16, name='x')
  Keyword arguments: {'training': False}

Option 2:
  Positional arguments (1 total):
    * TensorSpec(shape=(None, 352, 352, 3), dtype=tf.float16, name='x')
  Keyword arguments: {'training': True}

Option 3:
  Positional arguments (1 total):
    * TensorSpec(shape=(None, 352, 352, 3), dtype=tf.float16, name='input_1')
  Keyword arguments: {'training': False}

Option 4:
  Positional arguments (1 total):
    * TensorSpec(shape=(None, 352, 352, 3), dtype=tf.float16, name='input_1')
  Keyword arguments: {'training': True}